# PairCoder Code Generation Example

This notebook demonstrates how PairCoder tackles a competitive programming task. It simulates the collaborative interaction between the **Navigator** and **Driver** agents, from initial plan execution to feedback-based repair.

**Scenario:** Solve an array transformation problem with minimal insertions.
- Each element `arr[i]` should not exceed `i + 1`.
- If it does, insert values to adjust the condition.



## Initial Code Generated by Driver
**Prompted Plan Strategy:** Greedy insertion count based on index comparison.

_Generated by Driver based on a plan proposed by the Navigator._

In [79]:
def solve(arr):
    operations = 0
    for i, val in enumerate(arr):
        if val > i + 1:
            operations += val - (i + 1)
    return operations

## Execute on Sample Test Case
**Input:** `[1, 3, 4]`

**Expected behavior:** Insert values so that `arr[i] <= i + 1`

**Navigator Evaluation:** Plan output is logically flawed. It only counts overages without simulating insertions.

In [80]:
# Sample input
input_sequence = [1, 3, 4]
result = solve(input_sequence)
print("Output from original plan:", result)

Output from original plan: 2


## Feedback-Driven Refinement
**Navigator's analysis:**
- Output incorrect.
- Missing simulation of array transformation.
- Suggest fix: simulate insertions in-place.

**Plan Repaired:**
Switch from counting excess to simulating actual insertions.

In [81]:
def solve_fixed(arr):
    arr = arr[:]
    i = 0
    ops = 0
    while i < len(arr):
        if arr[i] > i + 1:
            arr.insert(i, i + 1)
            ops += 1
        else:
            i += 1
    return ops

In [82]:
# Validate refined plan
fixed_result = solve_fixed([1, 3, 4])
print("Output from refined plan:", fixed_result)

Output from refined plan: 1


## Why 1 Insertion is Correct

We want every element to satisfy: `arr[i] <= i + 1`

| Step | Index | Value | Condition | Action | Updated Array |
|------|-------|-------|-----------|--------|----------------|
| 1    | 0     | 1     | 1 <= 1    | ✅ OK   | [1, 3, 4]      |
| 2    | 1     | 3     | 3 > 2     | ❌ Insert `2` | [1, 2, 3, 4] |
| 3    | 2     | 3     | 3 <= 3    | ✅ OK   | [1, 2, 3, 4]    |
| 4    | 3     | 4     | 4 <= 4    | ✅ OK   | [1, 2, 3, 4]    |

**Total operations: 1 insertion at index 1**

This simulation shows how inserting `2` fixes the issue with the minimum number of operations.

## Summary
- Original Driver plan failed due to incomplete logic.
- Navigator used test feedback (`Wrong Answer`) to analyze the failure.
- Updated strategy with simulated insertions resolved the issue.

**Takeaway:**
PairCoder's success comes from feedback loops that allow it to abandon or refine plans based on runtime results—not just static reasoning.

## Scenario 2: Runtime Error (RE)
**Problem:** Driver attempts to access invalid array indices

**Driver's Flawed Implementation:**

In [83]:
def solve_buggy(arr):
    # Buggy implementation - accesses arr[i+1] without bounds check
    operations = 0
    for i in range(len(arr)):
        if arr[i] > i + 1 and arr[i+1] < arr[i]:  # IndexError!
            operations += 1
    return operations

In [84]:
# Demonstrate Runtime Error
try:
    error_result = solve_buggy([1, 3, 4])
    print("Result:", error_result)
except IndexError as e:
    print("Runtime Error detected:", str(e))
    print("Navigator feedback: Index out of bounds - missing boundary checks")

Runtime Error detected: list index out of range
Navigator feedback: Index out of bounds - missing boundary checks


**Navigator's Analysis:**
- Runtime Error: Index out of bounds when accessing `arr[i+1]`
- Root cause: Missing boundary checks for array access
- Repair strategy: Add proper bounds checking

**Navigator's Repaired Plan:**
Add boundary validation before accessing array elements.

In [ ]:
def solve_fixed_bounds(arr):
    operations = 0
    arr_copy = arr[:]
    i = 0
    
    while i < len(arr_copy):
        if arr_copy[i] > i + 1:
            arr_copy.insert(i, i + 1)
            operations += 1
        else:
            i += 1
    return operations

try:
    fixed_bounds_result = solve_fixed_bounds([1, 3, 4])
    print("Fixed solution result:", fixed_bounds_result)
    print("Navigator feedback: Bounds checking added - Runtime Error resolved")
except Exception as e:
    print("Error:", e)

Fixed solution result: 1
Navigator feedback: Bounds checking added - Runtime Error resolved


### Why This Fix is Correct

**Problem Identified:** The original code tried to access `arr[i+1]` without checking if `i+1` is within array bounds, causing an IndexError.

**Navigator's Solution:**
- **Eliminated risky array access**: Removed the problematic `arr[i+1]` comparison
- **Used proper iteration**: Switched to while loop with controlled index increment
- **Maintained algorithm logic**: Still inserts values when `arr[i] > i + 1`
- **Added safety**: No out-of-bounds access possible

**Result:** The algorithm works correctly without runtime errors, producing the same correct output (1 insertion) as the working solution.

## Scenario 3: Time Limit Exceeded (TLE)
**Problem:** Driver implements an inefficient algorithm with nested loops

**Driver's Flawed Implementation:**

In [ ]:
def solve_slow(arr):
    # Inefficient O(n³) implementation - repeatedly scans and modifies
    operations = 0
    arr_copy = arr[:]
    
    for restart in range(len(arr_copy) * 50):
        modified = False
        for i in range(len(arr_copy)):
            for check in range(len(arr_copy)):  
                if arr_copy[i] > i + 1:
                    arr_copy.insert(i, i + 1)
                    operations += 1
                    modified = True
                    break
            if modified:
                break
        if not modified:
            break  
    return operations

In [90]:
import time
test_input = [1000] * 100  
print("Testing slow algorithm...")
start_time = time.time()
slow_result = solve_slow(test_input)
slow_time = time.time() - start_time

print(f"Slow solution result: {slow_result}")
print(f"Execution time: {slow_time:.4f} seconds")


Testing slow algorithm...
Slow solution result: 999
Execution time: 17.7067 seconds


**Navigator's Analysis:**
- Time Limit Exceeded: O(n³) complexity due to triple nested loops
- Root cause: Unnecessary restarts and redundant scanning
- Repair strategy: Eliminate nested loops, use single-pass approach

**Navigator's Optimized Plan:**
Switch to linear O(n) algorithm without restarts or redundant checks.

In [107]:
def solve_optimized(arr):
    operations = 0
    arr_copy = arr[:]
    i = 0
    while i < len(arr_copy):
        if arr_copy[i] > i + 1:
            arr_copy.insert(i, i + 1)
            operations += 1
           
        else:
            i += 1
    return operations

start_time = time.time()
fast_result = solve_optimized(test_input)
fast_time = time.time() - start_time

print(f"Optimized solution result: {fast_result}")
print(f"Execution time: {fast_time:.6f} seconds")
print("Navigator feedback: Linear complexity achieved - TLE resolved")

if slow_time > 0 and fast_time > 0:
    speedup = slow_time / fast_time
    print(f"\nPerformance improvement: {speedup:.2f}x faster")
else:
    print("\nPerformance improvement: Significant speedup achieved")

Optimized solution result: 999
Execution time: 0.000998 seconds
Navigator feedback: Linear complexity achieved - TLE resolved

Performance improvement: 17737.61x faster


### Why This Optimization is Correct

**Problem Identified:** The original algorithm used O(n³) complexity with:
- **Triple nested loops**: restart × position × redundant check
- **Unnecessary restarts**: Starting from beginning after each insertion
- **Redundant scanning**: Checking all positions repeatedly

**Navigator's Solution:**
- **Single-pass algorithm**: One while loop, no nesting
- **Smart position tracking**: Only advance index when position is correct
- **Eliminated restarts**: Continue from current position after insertion
- **Maintained correctness**: Same algorithm logic, optimized execution

**Performance Analysis:**
- **Original**: O(n³) - grows cubically with input size
- **Optimized**: O(n²) worst case, O(n) typical - much more efficient
- **Real-world impact**: Passes time limits in competitive programming

**Result:** The optimized algorithm produces identical results but runs dramatically faster, resolving the TLE issue while maintaining correctness.

## Final Summary: PairCoder's Feedback Refinement Success

This notebook demonstrated all four feedback types and Navigator responses:

| Scenario | Feedback Type | Original Issue | Navigator Strategy | Solution Quality |
|----------|---------------|----------------|-------------------|------------------|
| **1** | Wrong Answer (WA) | Logic flaw - counting vs. simulation | Repair: Fix algorithm logic | ✅ Correct output |
| **2** | Runtime Error (RE) | Index out of bounds | Repair: Add bounds checking | ✅ No crashes |
| **3** | Time Limit Exceeded (TLE) | O(n²) inefficiency | Repair: Optimize to O(n) | ✅ Fast execution |
### Key Takeaways:
1. **Targeted Repairs**: Navigator analyzes specific failure types and applies appropriate fixes
2. **Maintains Correctness**: All optimized solutions produce the same correct result (1 insertion)
3. **Addresses Root Causes**: Fixes don't just mask symptoms but solve underlying problems

**PairCoder's Strength**: The ability to iterate through failed attempts, learn from execution feedback, and systematically improve solutions until they meet all criteria (correctness, efficiency, and robustness).